In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from glob import glob
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import xarray as xr
import rasterio as rio
import rioxarray

import crunchy_snow.models
import crunchy_snow.dataset

In [2]:
data_fn = '../../data/application_tmp/RMNP.nc'
ds = xr.open_dataset(data_fn)

In [4]:
ds = ds.fillna(0)

In [8]:
norm_ds = ds.copy()

In [13]:
ds

<xarray.Dataset>
Dimensions:      (y: 1542, x: 728)
Coordinates:
  * y            (y) float64 4.499e+06 4.499e+06 ... 4.422e+06 4.422e+06
  * x            (x) float64 4.257e+05 4.257e+05 4.258e+05 ... 4.62e+05 4.62e+05
    band         int64 ...
Data variables: (12/31)
    snowon_vh    (y, x) float32 -19.46 -19.22 -16.88 ... -18.77 -18.74 -17.31
    snowon_vv    (y, x) float32 -13.31 -13.09 -10.77 ... -11.49 -12.77 -10.86
    spatial_ref  int64 0
    snowoff_vh   (y, x) float32 -17.35 -17.74 -15.57 ... -17.33 -17.9 -15.97
    snowoff_vv   (y, x) float32 -10.94 -12.05 -9.152 ... -10.88 -10.2 -8.654
    AOT          (y, x) float32 73.0 73.0 73.0 73.0 ... 116.5 117.0 117.0 117.0
    ...           ...
    delta_cr     (y, x) float32 0.259 -0.4501 0.3138 ... -0.8329 1.729 0.866
    ndvi         (y, x) float32 -0.03336 0.002072 -0.02524 ... 0.1426 0.09959
    ndsi         (y, x) float32 0.5958 0.6466 0.5854 ... 0.2047 0.1042 0.1368
    ndwi         (y, x) float32 0.02797 0.02292 0.03801 ... -0.1726 -0.0975
    latitude     (y, x) float64 40.64 40.64 40.64 40.64 ... 39.94 39.94 39.94
    longitude    (y, x) float64 -105.9 -105.9 -105.9 ... -105.4 -105.4 -105.4

In [ ]:
# normalize layers (except gap maps and fcf)
norm_ds['snowon_vv'] = torch.clamp(calc_norm(ds['snowon_vv'], crunchy_snow.datasetnorm_dict['vv']), 0, 1)
norm_ds['snowon_vh'] = torch.clamp(calc_norm(ds['snowon_vh'], crunchy_snow.dataset.norm_dict['vh']), 0, 1)
norm_ds['snowoff_vv'] = torch.clamp(calc_norm(ds['snowoff_vv'], crunchy_snow.dataset.norm_dict['vv']), 0, 1)
norm_ds['snowoff_vh'] = torch.clamp(calc_norm(ds['snowoff_vh'], crunchy_snow.dataset.norm_dict['vh']), 0, 1)
norm_ds['aerosol_optical_thickness'] = torch.clamp(calc_norm(ds['AOT'], crunchy_snow.dataset.norm_dict['AOT']), 0, 1)
norm_ds['coastal_aerosol'] = torch.clamp(calc_norm(ds['B01'], crunchy_snow.dataset.norm_dict['coastal']), 0, 1)
norm_ds['blue'] = torch.clamp(calc_norm(ds['B02'], crunchy_snow.dataset.norm_dict['blue']), 0, 1)
norm_ds['green'] = torch.clamp(calc_norm(green, crunchy_snow.dataset.norm_dict['green']), 0, 1)
norm_ds['red'] = torch.clamp(calc_norm(red, crunchy_snow.dataset.norm_dict['red']), 0, 1)
norm_ds['red_edge1'] = torch.clamp(calc_norm(red_edge1, crunchy_snow.dataset.norm_dict['red_edge1']), 0, 1)
norm_ds['red_edge2'] = torch.clamp(calc_norm(red_edge2, crunchy_snow.dataset.norm_dict['red_edge2']), 0, 1)
norm_ds['red_edge3'] = torch.clamp(calc_norm(red_edge3, crunchy_snow.dataset.norm_dict['red_edge3']), 0, 1)
norm_ds['nir'] = torch.clamp(calc_norm(nir, crunchy_snow.dataset.norm_dict['nir']), 0, 1)
norm_ds['water_vapor'] = torch.clamp(calc_norm(water_vapor, crunchy_snow.dataset.norm_dict['water_vapor']), 0, 1)
norm_ds['swir1'] = torch.clamp(calc_norm(swir1, crunchy_snow.dataset.norm_dict['swir1']), 0, 1)
norm_ds['swir2'] = torch.clamp(calc_norm(swir2, crunchy_snow.dataset.norm_dict['swir2']), 0, 1)
norm_ds['scene_class_map'] = torch.clamp(calc_norm(scene_class_map, crunchy_snow.dataset.norm_dict['scene_class_map']), 0, 1)
norm_ds['water_vapor_product'] = torch.clamp(calc_norm(water_vapor_product, crunchy_snow.dataset.norm_dict['water_vapor_product']), 0, 1)
norm_ds['elevation'] = torch.clamp(calc_norm(elevation, crunchy_snow.dataset.norm_dict['elevation']), 0, 1)
norm_ds['latitude'] = torch.clamp(calc_norm(latitude, crunchy_snow.dataset.norm_dict['latitude']), 0, 1)
norm_ds['longitude'] = torch.clamp(calc_norm(longitude, crunchy_snow.dataset.norm_dict['longitude']), 0, 1)
# norm_ds['dowy'] = torch.clamp(torch.nan_to_num(calc_norm(dowy, [0, 365]), 0), 0, 1)
norm_ds['ndvi'] = torch.clamp(torch.nan_to_num(calc_norm(ndvi, [-1, 1]), 0), 0, 1)
norm_ds['ndsi'] = torch.clamp(torch.nan_to_num(calc_norm(ndsi, [-1, 1]), 0), 0, 1)
norm_ds['ndwi'] = torch.clamp(torch.nan_to_num(calc_norm(ndwi, [-1, 1]), 0), 0, 1)
norm_ds['snowon_cr'] = torch.clamp(torch.nan_to_num(calc_norm(snowon_cr, crunchy_snow.dataset.norm_dict['cr']), 0), 0, 1)
norm_ds['snowoff_cr'] = torch.clamp(torch.nan_to_num(calc_norm(snowoff_cr, crunchy_snow.dataset.norm_dict['cr']), 0), 0, 1)
norm_ds['delta_cr'] = torch.clamp(torch.nan_to_num(calc_norm(delta_cr, crunchy_snow.dataset.norm_dict['delta_cr']), 0), 0, 1)

In [ ]:
# these are set by finding the min and max across the entire dataset
norm_dict = {'aso_sd':[0, 25],
             'vv':[-59, 42],
             'vh':[-70, 19],
             'cr':[-43, 16],
             'delta_cr':[-33, 27],
             'AOT':[0, 572],
             'coastal':[0, 24304],
             'blue':[0, 23371],
             'green':[0, 26440],
             'red':[0, 21576],
             'red_edge1':[0, 20796],
             'red_edge2':[0, 20432],
             'red_edge3':[0, 20149],
             'nir':[0, 21217],
             'water_vapor':[0, 18199],
             'swir1':[0, 17669],
             'swir2':[0, 17936],
             'scene_class_map':[0, 15],
             'water_vapor_product':[0, 6518],
             'elevation':[-100, 9000],
             'latitude':[-90, 90],
             'longitude':[-180, 180]}

In [6]:
ds

<xarray.Dataset>
Dimensions:      (y: 1542, x: 728)
Coordinates:
  * y            (y) float64 4.499e+06 4.499e+06 ... 4.422e+06 4.422e+06
  * x            (x) float64 4.257e+05 4.257e+05 4.258e+05 ... 4.62e+05 4.62e+05
    band         int64 ...
Data variables: (12/31)
    snowon_vh    (y, x) float32 -19.46 -19.22 -16.88 ... -18.77 -18.74 -17.31
    snowon_vv    (y, x) float32 -13.31 -13.09 -10.77 ... -11.49 -12.77 -10.86
    spatial_ref  int64 0
    snowoff_vh   (y, x) float32 -17.35 -17.74 -15.57 ... -17.33 -17.9 -15.97
    snowoff_vv   (y, x) float32 -10.94 -12.05 -9.152 ... -10.88 -10.2 -8.654
    AOT          (y, x) float32 73.0 73.0 73.0 73.0 ... 116.5 117.0 117.0 117.0
    ...           ...
    delta_cr     (y, x) float32 0.259 -0.4501 0.3138 ... -0.8329 1.729 0.866
    ndvi         (y, x) float32 -0.03336 0.002072 -0.02524 ... 0.1426 0.09959
    ndsi         (y, x) float32 0.5958 0.6466 0.5854 ... 0.2047 0.1042 0.1368
    ndwi         (y, x) float32 0.02797 0.02292 0.03801 ... -0.1726 -0.0975
    latitude     (y, x) float64 40.64 40.64 40.64 40.64 ... 39.94 39.94 39.94
    longitude    (y, x) float64 -105.9 -105.9 -105.9 ... -105.4 -105.4 -105.4

In [5]:
crunchy_snow.dataset.calc_norm()

TypeError: calc_norm() missing 2 required positional arguments: 'tensor' and 'minmax_list'

In [ ]:
#load previous model
model = crunchy_snow.models.ResDepth(n_input_channels=len(input_channels))
model.load_state_dict(torch.load('../../weights/quinn_ResDepth_v3'))
model.to('cuda');